# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [94]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql import Row
from pyspark.sql.functions import sum as Fsum
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, DateType
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.sql import Window
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
import datetime
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, LinearSVC, GBTClassifier

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics


In [47]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [48]:
df = spark.read.json("mini_sparkify_event_data.json")

In [49]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [50]:
#total record counts
df.count()

286500

In [51]:
#remove the data having NaNs for sessionId or userId
valid_user = df.dropna(how = "any", subset = ["userId","sessionId"])

In [52]:
#record counts after removing bad records
valid_user.count()

286500

In [53]:
#check if the userId is null present
valid_user.select("userId").dropDuplicates().sort("userId").show()

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
+------+
only showing top 20 rows



We can see the first record is having userId as null

In [54]:
#let's remove the null userId from the records
valid_user = valid_user.filter(valid_user['userId'] != "")

In [55]:
#let's do the count check on number of records after removing nulls from user id
valid_user.count()

278154

This count shows there are some records deleted from the valid_user

In [56]:
#see some sample records
valid_user.show(5)

+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|          artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|  Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Boston-Cambridge-...|   PUT|NextSong|1538331630000| 

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [57]:
#lets explore the pages available 
valid_user.select("page").dropDuplicates().sort("page").show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|              Logout|
|            NextSong|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
|      Submit Upgrade|
|         Thumbs Down|
|           Thumbs Up|
|             Upgrade|
+--------------------+



In [58]:
df_pd = valid_user.toPandas()

In [59]:
df_pd.shape

(278154, 18)

In [60]:
valid_user.filter("page = 'Cancellation Confirmation'").show()

+------+---------+---------+------+-------------+---------+------+-----+--------------------+------+--------------------+-------------+---------+----+------+-------------+--------------------+------+
|artist|     auth|firstName|gender|itemInSession| lastName|length|level|            location|method|                page| registration|sessionId|song|status|           ts|           userAgent|userId|
+------+---------+---------+------+-------------+---------+------+-----+--------------------+------+--------------------+-------------+---------+----+------+-------------+--------------------+------+
|  null|Cancelled|   Adriel|     M|          104|  Mendoza|  null| paid|  Kansas City, MO-KS|   GET|Cancellation Conf...|1535623466000|      514|null|   200|1538943990000|"Mozilla/5.0 (Mac...|    18|
|  null|Cancelled|    Diego|     M|           56|    Mckee|  null| paid|Phoenix-Mesa-Scot...|   GET|Cancellation Conf...|1537167593000|      540|null|   200|1539033046000|"Mozilla/5.0 (iPh...|    32|


In [61]:
df_copy = valid_user

In [62]:
#create a Churn column with value as 0 or 1
cancellation_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
valid_user = valid_user.withColumn("Churn", cancellation_event("page"))
valid_user.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', Churn=0)

In [63]:
valid_user.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- Churn: integer (nullable = true)



As from the above list we can see there are multiple columns which has type string. As to train with our model we need to convert columns from string to numbers or integers.

In [64]:
churn = udf(lambda x: int(x=="Cancellation Confirmation"), IntegerType())
downgrade_churn = udf(lambda x: int(x=="Submit Downgrade"), IntegerType())

df_copy = df_copy.withColumn("downgraded", downgrade_churn("page")).withColumn("cancelled", churn("page"))

In [65]:
df_copy.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- downgraded: integer (nullable = true)
 |-- cancelled: integer (nullable = true)



In [66]:
#distribution of users downgrades and cancellations
df_copy.select(['userId', 'downgraded', 'cancelled'])\
    .groupBy('userId').sum()\
    .withColumnRenamed('sum(downgraded)', 'Downgraded')\
    .withColumnRenamed('sum(cancelled)', 'Cancelled').describe().show()

+-------+------------------+------------------+------------------+
|summary|            userId|        Downgraded|         Cancelled|
+-------+------------------+------------------+------------------+
|  count|               225|               225|               225|
|   mean|65391.013333333336|              0.28|0.2311111111111111|
| stddev|105396.47791907164|0.5876709477736184|0.4224832108996327|
|    min|                10|                 0|                 0|
|    max|                99|                 3|                 1|
+-------+------------------+------------------+------------------+



In [67]:
windowvalue = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)

df_copy = df_copy.withColumn("churn", Fsum("cancelled").over(windowvalue))\
    .withColumn("downgrade_phase", Fsum("downgraded").over(windowvalue))

In [68]:
df_copy.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- downgraded: integer (nullable = true)
 |-- cancelled: integer (nullable = true)
 |-- churn: long (nullable = true)
 |-- downgrade_phase: long (nullable = true)



In [69]:
song=udf(lambda x : int(x=='NextSong'), IntegerType())
home_visit=udf(lambda x : int(x=='Home'), IntegerType())
get_day = udf(lambda x: datetime.datetime.fromtimestamp(x/1000), DateType())
df_copy = df_copy.withColumn('date', get_day(col('ts')))

In [70]:
#difference in number of songs played between users who churned and didnt?
df_copy.filter(col('churn')==1).withColumn('songPlayed', song(col('page'))).agg({'songPlayed':'mean'}).show()
df_copy.filter(col('churn')==0).withColumn('songPlayed', song(col('page'))).agg({'songPlayed':'mean'}).show()

+------------------+
|   avg(songPlayed)|
+------------------+
|0.8112072039942939|
+------------------+

+------------------+
|   avg(songPlayed)|
+------------------+
|0.8217840456084702|
+------------------+



In [71]:
days = lambda i: i * 86400 
daywindow = Window.partitionBy('userId', 'date').orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding, 0)
get_day = udf(lambda x: datetime.datetime.fromtimestamp(x/1000), DateType())

In [72]:
print("Number of songs played daily")
df_copy.filter((df_copy.page=='NextSong')&(col('churn')==1)).select('userId', 'page', 'ts')\
    .withColumn('date', get_day(col('ts'))).groupBy('userId', 'date').count().describe().show()

df_copy.filter((df_copy.page=='NextSong')&(col('churn')==1)).select('userId', 'page', 'ts')\
    .withColumn('date', get_day(col('ts'))).groupBy('userId', 'date').count().describe().show()

Number of songs played daily
+-------+-----------------+-----------------+
|summary|           userId|            count|
+-------+-----------------+-----------------+
|  count|              499|              499|
|   mean|77394.81563126252|72.93386773547094|
| stddev|90869.89716037885|71.24764235703725|
|    min|           100001|                1|
|    max|               87|              346|
+-------+-----------------+-----------------+

+-------+-----------------+-----------------+
|summary|           userId|            count|
+-------+-----------------+-----------------+
|  count|              499|              499|
|   mean|77394.81563126252|72.93386773547094|
| stddev|90869.89716037885|71.24764235703725|
|    min|           100001|                1|
|    max|               87|              346|
+-------+-----------------+-----------------+



In [73]:
df_copy.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- downgraded: integer (nullable = true)
 |-- cancelled: integer (nullable = true)
 |-- churn: long (nullable = true)
 |-- downgrade_phase: long (nullable = true)
 |-- date: date (nullable = true)



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [74]:
#pick gender as first feature
df_gender = df_copy.dropDuplicates(['userId']).sort('userId').select(['userId','gender'])
df_gender = df_gender.replace(['F','M'], ['1', '0'], 'gender')
df_gender.withColumn('gender', df_gender.gender.cast("int"))

DataFrame[userId: string, gender: int]

In [75]:
#pick number of songs played as second feature
df_songs = df_copy.where(df_copy.song!='null').groupby('userId') \
    .agg(count(df_copy.song).alias('SongsPlayed')).orderBy('userId') \
    .select(['userId','SongsPlayed'])

In [76]:
df_songs.show(2)

+------+-----------+
|userId|SongsPlayed|
+------+-----------+
|    10|        673|
|   100|       2682|
+------+-----------+
only showing top 2 rows



In [77]:
#pick number of thumbs up and thumps down as third and fourth feature
df_thumbsup = df_copy.where(df_copy.page=='Thumbs Up') \
    .groupby(['userId']).agg(count(col('page')).alias('ThumbsUps')).orderBy('userId').select(['userId','ThumbsUps'])

df_thumbsdown = df_copy.where(df_copy.page=='Thumbs Down') \
    .groupby(['userId']).agg(count(col('page')).alias('ThumbsDowns')).orderBy('userId').select(['userId','ThumbsDowns'])

In [78]:
df_thumbsup.show(2)

+------+---------+
|userId|ThumbsUps|
+------+---------+
|    10|       37|
|   100|      148|
+------+---------+
only showing top 2 rows



In [79]:
df_thumbsdown.show(2)

+------+-----------+
|userId|ThumbsDowns|
+------+-----------+
|    10|          4|
|   100|         27|
+------+-----------+
only showing top 2 rows



In [80]:
#pick the number of singers user has listened as fifth feature
df_singers = df_copy.dropDuplicates(['userId','artist']).groupby('userId') \
    .agg(count(df_copy.artist).alias('SingersListened')).orderBy('userId') \
    .select(['userId','SingersListened'])

In [81]:
df_singers.show(2)

+------+---------------+
|userId|SingersListened|
+------+---------------+
|    10|            565|
|   100|           1705|
+------+---------------+
only showing top 2 rows



In [82]:
#join together different features
df_final = df_copy.dropDuplicates(['userId']).sort('userId').select(['userId','churn'])
for feature in [df_gender, df_songs,df_thumbsup,df_thumbsdown,df_singers]:
    df_final = df_final.join(feature,'userId')

In [83]:
df_final.show()

+------+-----+------+-----------+---------+-----------+---------------+
|userId|churn|gender|SongsPlayed|ThumbsUps|ThumbsDowns|SingersListened|
+------+-----+------+-----------+---------+-----------+---------------+
|100010|    0|     1|        275|       17|          5|            252|
|200002|    0|     0|        387|       21|          6|            339|
|   124|    0|     1|       4079|      171|         41|           2232|
|    51|    1|     0|       2111|      100|         21|           1385|
|     7|    0|     0|        150|        7|          1|            142|
|    15|    0|     0|       1914|       81|         14|           1302|
|    54|    1|     1|       2841|      163|         29|           1744|
|   155|    0|     1|        820|       58|          3|            643|
|100014|    1|     0|        257|       17|          3|            233|
|   132|    0|     1|       1928|       96|         17|           1299|
|   101|    1|     0|       1797|       86|         16|         

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [84]:
df_final.groupby("churn").count().show()

+-----+-----+
|churn|count|
+-----+-----+
|    0|  155|
|    1|   47|
+-----+-----+



In [85]:
df_final.dtypes

[('userId', 'string'),
 ('churn', 'bigint'),
 ('gender', 'string'),
 ('SongsPlayed', 'bigint'),
 ('ThumbsUps', 'bigint'),
 ('ThumbsDowns', 'bigint'),
 ('SingersListened', 'bigint')]

In [86]:
# convert data type of columns into float
for feature in df_final.columns[1:]:
    df_final = df_final.withColumn(feature,df_final[feature].cast('float'))

In [87]:
df_final.dtypes

[('userId', 'string'),
 ('churn', 'float'),
 ('gender', 'float'),
 ('SongsPlayed', 'float'),
 ('ThumbsUps', 'float'),
 ('ThumbsDowns', 'float'),
 ('SingersListened', 'float')]

In [88]:
df_final.show(5)

+------+-----+------+-----------+---------+-----------+---------------+
|userId|churn|gender|SongsPlayed|ThumbsUps|ThumbsDowns|SingersListened|
+------+-----+------+-----------+---------+-----------+---------------+
|100010|  0.0|   1.0|      275.0|     17.0|        5.0|          252.0|
|200002|  0.0|   0.0|      387.0|     21.0|        6.0|          339.0|
|   124|  0.0|   1.0|     4079.0|    171.0|       41.0|         2232.0|
|    51|  1.0|   0.0|     2111.0|    100.0|       21.0|         1385.0|
|     7|  0.0|   0.0|      150.0|      7.0|        1.0|          142.0|
+------+-----+------+-----------+---------+-----------+---------------+
only showing top 5 rows



In [89]:
train, test = df_final.randomSplit([0.8, 0.2], seed=42)

In [92]:
#build the pipeline for the logistic regression model
assembler = VectorAssembler(inputCols=df_final.columns[2:],outputCol='featuresassemble')
scaler = StandardScaler(inputCol="featuresassemble", outputCol="features", withStd=True)
indexer = StringIndexer(inputCol="churn", outputCol="label")

lr =  LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0)

stringIndexer = StringIndexer(inputCol="label", outputCol="indexed")
rf = RandomForestClassifier(numTrees=3, maxDepth=2, labelCol="indexed", seed=42)

In [95]:
pipeline_lr = Pipeline(stages=[assembler, scaler, indexer, lr])

paramGrid_lr = ParamGridBuilder() \
    .addGrid(lr.regParam,[0.0, 0.1, 0.01]) \
    .build()


crossval_lr = CrossValidator(estimator=pipeline_lr,
                          estimatorParamMaps=paramGrid_lr,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

In [96]:
cvModel_lr = crossval_lr.fit(train)

A parameter grid was used to determine the best value of parameters. The various parameters that were chosen was 0.0, 0.1 and 0.01. I did not choose a large amount of parameters to begin with to quickly determine if I was heading in the right direction or not. I fed the these parameters into the Cross Validator object along with the pipeline and the evaluator.

In [97]:
# test performance
results = cvModel_lr.transform(test)
BinaryClassificationEvaluator().evaluate(results)

0.7238095238095238

In [100]:
print(results.filter(results.label == results.prediction).count())
print(results.count())

21
31


In [102]:
21/31

0.6774193548387096

When the model was finished training, I have used cross validator average metrics to get the accuracy of both training and testing set. Here I have choosen only three parameters here and result is not bad as we can say by seeing the result that model is not overfitting.

I am happy to work on this project but I will also try to work on the entire data set. This should be give me an idea about how realtime project operates. Here I have not tried with multiple algorithm and idea here is to pass this course, but for learning opportunity I will explore working on 12GB dataset..

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.